In [28]:
import os
from typing import Annotated, TypedDict
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import InMemorySaver

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

search_tool = TavilySearchResults(k=2)
tools = [search_tool]

llm = ChatGroq(model="qwen/qwen3-32b")
llm_with_tools = llm.bind_tools(tools)

In [29]:
def chatbot(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [30]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", chatbot)
workflow.add_node("tools", ToolNode(tools))

workflow.add_edge(START, "agent")
# Conditional edge: if LLM wants tools, go to 'tools', else END
workflow.add_conditional_edges("agent", tools_condition)
workflow.add_edge("tools", "agent")


In [31]:
# 5. CRITICAL: Add memory and the Breakpoint
# We interrupt BEFORE the "tools" node is executed.
memory = InMemorySaver()
app = workflow.compile(checkpointer=memory, interrupt_before=["tools"])

In [ ]:
# --- RUNNING THE GRAPH ---
config = {"configurable": {"thread_id": "1"}}
user_input = "What is the current price of Bitcoin?"

print("--- Starting the Agent ---")
events = app.stream({"messages": [("user", user_input)]}, config, stream_mode="values")

for event in events:
    if "messages" in event:
        event["messages"].pretty_print()

# Check if we are paused
snapshot = app.get_state(config)
if snapshot.next:
    print(f"\nPAUSED: The agent wants to call: {snapshot.next}")
    print("The tool call is waiting for your approval...")
    
    # Human Input Simulation
    confirm = input("Type 'yes' to allow the search, or 'no' to stop: ")
    
    if confirm.lower() == 'yes':
        print("\n--- Resuming Search ---")
        # Passing None resumes from the checkpoint
        for event in app.stream(None, config, stream_mode="values"):
            if "messages" in event:
                event["messages"][-1].pretty_print()
    else:
        print("Search cancelled by human.")

--- Starting the Agent ---
================================ Human Message =================================

What is the current price of Bitcoin?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (2setbshf2)
 Call ID: 2setbshf2
  Args:
    query: Bitcoin price

PAUSED: The agent wants to call: ('tools',)
The tool call is waiting for your approval...

--- Resuming Search ---
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (2setbshf2)
 Call ID: 2setbshf2
  Args:
    query: Bitcoin price
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Bitcoin price today, BTC to USD live price, marketcap and chart", "url": "https://coinmarketcap.com/currencies/bitcoin/", "content": "16.86%\n\nAxelar\n\n25.73%\n\n### Bitcoin Price Live Data\n\nThe live Bitcoin price today is $66,382.9